# Prevendo fraudes de cartão de crédito

A fraude de cartão de crédito é uma das principais causas de roubo de identidade em todo o mundo. As instituições financeiras empregam uma grande variedade de técnicas diferentes para prevenir fraudes, sendo uma das mais comuns a Regressão Logística.

Neste projeto, você é um Cientista de Dados que trabalha para uma empresa de cartão de crédito. Você tem acesso a um conjunto de dados, que representa um conjunto típico de transações de cartão de crédito. Sua tarefa é usar a regressão logística e criar um modelo preditivo para determinar se uma transação é fraudulenta ou não.

In [7]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Carregar os dados

**1.** O arquivo `Transações.csv` contém dados de 200 mil transações de cartão de crédito simuladas. Comece carregando os dados em um DataFrame pandas chamado `transacoes`. Imprima algumas linhas e a contagem total de linhas. 

In [8]:
df = pd.read_csv('Transações.csv')
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,8,CASH_OUT,158007.12,C424875646,0.00,0.00,C1298177219,474016.32,1618631.97,0
1,236,CASH_OUT,457948.30,C1342616552,0.00,0.00,C1323169990,2720411.37,3178359.67,0
2,37,CASH_IN,153602.99,C900876541,11160428.67,11314031.67,C608741097,3274930.56,3121327.56,0
3,331,CASH_OUT,49555.14,C177696810,10865.00,0.00,C462716348,0.00,49555.14,0
4,250,CASH_OUT,29648.02,C788941490,0.00,0.00,C1971700992,56933.09,86581.10,0


### Limpe os dados

**2.** Olhando para o conjunto de dados, combinado com nosso conhecimento de transações de cartão de crédito em geral, podemos ver que existem algumas colunas interessantes para serem observadas. Sabemos que a coluna `amount`(quantia) de uma determinada transação será importante. Calcule estatísticas de resumo para esta coluna. Como é a distribuição?

In [16]:
df['amount'].describe()

count    1.999990e+05
mean     1.802425e+05
std      6.255482e+05
min      0.000000e+00
25%      1.338746e+04
50%      7.426695e+04
75%      2.086376e+05
max      5.204280e+07
Name: amount, dtype: float64

**3.** Temos muitas informações sobre o tipo (`type`) de transação que estamos analisando. Crie uma nova coluna chamada `é_Pagamento` que atribui um `1` quando `type` é “PAYMENT” ou “DEBIT” e `0` caso de outro tipo.

In [23]:
e_pagamento_cb = lambda type_value: 1 if type_value == 'PAYMENT' or type_value == 'DEBIT' else 0 

df['é_Pagamento'] = df['type'].apply(e_pagamento_cb) 

**4.** Da mesma forma, crie uma coluna chamada `é_movimentação`, que capturará se o dinheiro saiu da conta de origem. Esta coluna terá um valor de 1 quando `type` for “CASH_OUT” ou “TRANSFER”, e 0 caso contrário.

In [24]:
e_movimentacao_cb = lambda type_value: 1 if type_value == 'CASH_OUT' or type_value == 'TRANSFER' else 0 

df['é_movimentação'] = df['type'].apply(e_movimentacao_cb) 

**5.** Em fraudes financeiras, outro fator fundamental a ser investigado seria a diferença de valor entre a conta de origem e a conta de destino. A hipótese, neste caso, é que contas de destino com valor significativamente diferente podem ser suspeitas de fraude. Vamos criar uma coluna chamada `diferençaContas` com a diferença absoluta das colunas `oldbalanceOrge` e `oldbalanceDest`

### Selecionar e dividir os dados

**6.** Antes de começarmos a treinar nosso modelo, precisamos definir nossos recursos (variáveis de entrada) e a coluna que será a saída. A saída neste conjunto de dados é a coluna `isFraud`. Crie uma variável chamada `x` que será um array composto pelos seguintes campos:

- `amount`
- `é_Pagamento`
- `é_movimentação`
- `diferençaContas`

Crie também uma variável chamada `y` com a coluna `isFraud`.

**7.** Divida os dados em conjuntos de treinamento e teste usando o método `train_test_split()` do `sklearn`. Usaremos o conjunto de treinamento para treinar o modelo e o conjunto de teste para avaliar o modelo. Use um `test_size` de 0.3.

### Normalize os dados

**8.** No `sklearn` a implementação da regressão logística utiliza regularização e portanto precisamos dimencionar nossas variáveis de entrada. Crie um objeto `StandardScaler` , Utilize `.fit_transform()` nas variáveis de entrada do conjunto de treinamento e `.transform()` nas variáveis de entrada do conjunto de de teste.

### Criar e avaliar o modelo

**9.** Crie um modelo de regressão logística no `sklearn`.

**10.** Execute o método `.score()` do modelo nos dados de treinamento e imprima a pontuação de treinamento.

A pontuação do modelo nos dados de treinamento processará os dados de treinamento por meio do modelo treinado e preverá quais transações são fraudulentas. A pontuação retornada é a porcentagem de classificações corretas ou a precisão.

**11.** Imprima a matriz de confusão do modelo utilizando os dados do conjunto de teste.

**12.** Execute o método `.score()` do modelo nos dados de teste e imprima a pontuação do teste.

A pontuação do modelo nos dados de teste processará os dados de teste por meio do modelo treinado e preverá quais transações são fraudulentas. A pontuação retornada é a porcentagem de classificações corretas, ou a precisão, e será um indicador do sucesso do seu modelo.

**13.** Imprima os coeficientes do modelo para ver a importância de cada variável para a previsão. Qual característica foi mais importante? Menos importante?

**14.** Vamos usar o modelo criado para processar mais transações que passaram por nossos sistemas. Existem três matrizes numpy abaixo com uma amostra com informações sobre novas transações “Novos dados de transação”

Crie uma quarta matriz, `minha_transacao` e adicione qualquer informação de transação que desejar. Certifique-se de inserir todos os valores como `floats`.

In [9]:
# Novos dados de transação

transacao1 = np.array([123456.78, 0.0, 1.0, 54670.1])
transacao2 = np.array([98765.43, 1.0, 0.0, 8524.75])
transacao3 = np.array([543678.31, 1.0, 0.0, 510025.5])

# Crie dados de uma nova transação


**15.** Combine as novas transações e a matriz que você criou (`minha_transacao`) em uma única matriz numpy chamada `conjunto_transacoes`.

**16.** Como o modelo de regressão logística foi treinado com dados padronizados, também devemos patronizar os dados sobre os quais estamos fazendo previsões. Usando o objeto `StandardScaler`  criado anteriormente, aplique o método `.transform()` na matriz `conjunto_transacoes` e salve o resultado em `conjunto_transacoes`.

**17.** Quais transações são fraudulentas? Use o método `.predict()` do seu modelo na matriz `conjunto_transacoes` e imprima o resultado para descobrir.

Quer ver as probabilidades que levaram a essas previsões? Chame o método `.predict_proba()` do seu modelo e imprima o resultado. A 1ª coluna é a probabilidade de uma transação não ser fraudulenta e a 2ª coluna é a probabilidade de uma transação ser fraudulenta (que foi calculada pelo modelo para tomar a decisão final de classificação).